# Development practises


## Topics

- modules
- testing
- packages
- naming conventions
- pretty functions
- multiple dispatch
- variable scope
    - arrays as an exception
- metaprogramming (see Bonus notebook)

## Modules

Notebooks and the REPL are good for rapid development and testing things, but you may quickly find it limiting. Most of the time you want to develop any larger projects as modules and control scripts.

Modules are used to store function and structures you need often. They can be shared accross projects. The packages we use are modules written by others and shared so that we can use them and develop them further.

### Scripts

Julia code placed in a .jl file can be executed on the command line. We could place the ani

You can also `include()` the code like we did in the plotting exercises.

### Modules

A module is a collection of functions and data structures. Generally the function and structs are tied together by a single purpose or work are useful in similar situations.

To define a module called MyModule, you would create a file called `MyModule.jl` and write something like below:
```julia
module MyModule
export mycoolfunction

function mycoolfunction()
   println("this is my cool function")
end
function mysecretfunction()
   println("this is my secret function")
end

end
```

In fact this file is already in the currect directory.

In [ ]:
# To use the module you need to first include the file:

include("MyModule.jl")
using .MyModule

When you load the module with the `using` keyword, the `export`ed names are pulled into the global namespace.

In [ ]:
mycoolfunction()

Other names are not in global namespace, but they can still be used:

In [ ]:
# This will not work
mysecretfunction()

In [ ]:
# But this will
MyModule.mysecretfunction()

#### Import

Sometimes different modules define things with the same name. Multiple dispatch can take care of multiple methods with the same name, but only if they use different types of input.



In [ ]:
module MyOtherModule
export mycoolfunction

function mycoolfunction()
   println("This is my other module")
end
end

import Main.MyOtherModule

In [ ]:
mycoolfunction()

In [ ]:
MyOtherModule.mycoolfunction()

### include()

Using the include statement to spread your code is
into multiple files is a a common and useful pattern.

We could write the module above as

```julia
module MyModule
export mycoolfunction

inlude("mycoolfunction.jl")
inlude("mysecretfunction.jl")

end
```

Provided that the functions are defined in the given files.

Don't overdo it, though. In this case the functions are not complicated and there is not reason to have them in a separate file.

## Packages

Having to `include()` the file is a problem when you
want to use the same code across projects. 
For this we need to create a package.

A package is a folder with
 - One or more module files
 - a `Project.toml`

The `Project.toml` file contains some information about
the package and a list of other packages your package
depends on. The easiest way to go from a module file
(or several module files) to a pacakge is to use the
Pkg module:

In [ ]:
using Pkg
Pkg.generate("MyPackage")

This creates a new `MyPackage` folder with the Project.toml file
and a src directory for the module file. The src folder contains
an automatically generated example project.

Let's try it out. In the exercises you will create and edit a module, but here are satisfied with using the example package.

In [ ]:
Pkg.develop(PackageSpec(path="MyPackage"))
using MyPackage
MyPackage.greet()

`Pkg.develop` is similar to `Pkg.add`, but it any changes we make 
to the package will take effect when you import or use the package. With `Pkg.add` you would need to manually reinstall.

## Testing

- The only way to make sure your code actually works is to test it.
- Julia has convenient built-in macros for automated tests. 

In [ ]:
import Pkg
Pkg.add("Test")

In [ ]:
using Test
@test 1 + 1 == 2

- Group related tests together into a test set

In [ ]:
@testset "Test arithmetic equalities" begin
    @test 1 + 1 == 2
end

### Testing a package

It is good practice to add a set of tests to your packages. This
 - ensures that it works as intended
 - defines what actually works
 
The tests are added to `test/runtests.jl` in the package 
directory. We will add the following to `MyPackage`:
```julia
using Test
@testset "Test arithmetic equalities" begin
    @test 1 + 1 == 2
end
```

We also need to add the Test module to the our packages dependencies.
The `project.toml` file should look something like this:

```
name = "MyPackage"
uuid = "2079a258-defa-40a0-8316-dccef70ae21b"
authors = ["Jarno Rantaharju"]
version = "0.1.0"

[deps]
Test = "8dfed614-e22c-5e08-85e1-65c5234f0b40"
```

Now you can run the tests using the Pkg module:

In [ ]:
Pkg.test("MyPackage")

## Development environments and dependencies

The best way to track the packages you project depends on is to use 
Julia's built-in development environment manager.

When a development environment is active, Julia will only see the packages installed inside hat environment. The `project.toml` file is
automatically updated whenever you install a new package.

You can create an environment in any folder, it does not need to be a package. To do this, run
```Julia
Pkg.activate(".")
```

We can also activate the environment of our package by specifying its
path.

In [ ]:
Pkg.activate("MyPackage")
# Now the plot package is no longer available, since it is not listed
# in MyPackage/project.toml
using Plots

Now see what happens in when we install it:

In [ ]:
Pkg.add("Plots")

## Naming conventions in Julia

- Word separation can be indicated by underscores (`_`), but use of underscores is discouraged unless the name would be hard to read otherwise.
- Names of Types begin with a capital letter and word separation is shown with CamelCase instead of underscores.
- Names of functions and macros are in lower case, without underscores.  
- Functions that modify their inputs have names that end in `!`. These functions are sometimes called mutating functions or in-place functions.


## Making functions pretty: optional arguments
You can define functions with optional arguments, so that the function can use sensible defaults if specific values aren't supplied. You provide a default symbol and value in the argument list

In [ ]:
function xyzpos(x, y, z=0)
    println("$x, $y, $z")
end

In [ ]:
xyzpos(0,0)
xyzpos(0,0,1)

## Making functions pretty: keyword arguments
When you write a function with a long list of arguments like this:
```julia
function f(p, q, r, s, t, u)
...
end
```
sooner or later, you will forget the order in which you have to supply the arguments. 

You can avoid this problem by using keywords to label arguments. Use a semicolon (`;`) after the function's unlabelled arguments, and follow it with one or more keyword=value pairs:

In [ ]:
function f(p, q ; radius = 4, message = "hello")
  println("p is $p")
  println("q is $q")
  return "radius => $radius, message => $message"
end
f(1,2)
f("a", "b", r=pi, s=22//7)

## Advanced: Functions with variable number of arguments
Functions can be defined so that they can accept any number of arguments:

In [ ]:
function fvar(args...)
    println("you supplied $(length(args)) arguments")
    for arg in args
       println(" argument ", arg)
    end
end
fvar()
fvar(64)
fvar(64, 64, 55)

The three dots indicate the **splat**. Here it means "any", including "none". 

## Advanced: Parametric methods
Method definitions can optionally have type parameters qualifying the signature:

In [ ]:
function same_type(x::T, y::T) where {T}
    true
end

function same_type(x,y)
    false
end

The first method applies whenever both arguments are of the same concrete type, regardless of what type that is, while the second method acts as a catch-all, covering all other cases. Thus, overall, this defines a boolean function that checks whether its two arguments are of the same type

In [ ]:
same_type(1,2)

In [ ]:
same_type(1, 2.0)

In [ ]:
same_type(1.0, 2.0)

In [ ]:
same_type("foo", 2.0)

In [ ]:
same_type(Int32(1), Int64(2))

## Scope of variables
- Global scope
    - Module specific (namespaces)
- Local scopes
    - functions, for's, while's,...

## Local scope
A new local scope is introduced by most code-blocks.
    
A local scope usually inherits all the variables from its parent scope, both for reading and writing. 

A newly introduced variable in a local scope does not back-propagate to its parent scope. For example, here the z is not introduced into the top-level scope:

In [ ]:
for i = 1:10
    z = 1
end
z

Function definitions are also in their own local scope. 

They do, however, inherit from their parent scope.

In [ ]:
x, y = 1, 2
function foo()
    x = 2 #assignment introduces a new local
    return x + y # y refers to the global scope!
end

In [ ]:
foo()

In [ ]:
x

An explicit `global` is needed to assign to a global variable:

In [ ]:
x = 1
function foobar()
    global x = 2
    return x + y # Now x is also in global scope!
end

In [ ]:
foobar()

In [ ]:
x

## Exception: Elements of a global array are global
There is an important exception to these rules: arrays.

Changing an elements of an array in global scope is done in the global scope. An array points to a place in memory, so changing it's internal values changes the original memory, even inside a function.

In [ ]:
arr = [1,2,3]
function oops()
    arr[2] = 10
    arrr = [1,2,3]
    
    return "woops"
end

In [ ]:
oops()

In [ ]:
arr

In [ ]:
arrr

## Constants
A common use of variables is giving names to specific, unchanging values. 

Such variables are only assigned once. This intent can be conveyed to the compiler using the `const` keyword:

In [ ]:
const e  = 2.71828182845904523536

It is difficult for the compiler to optimize code involving global variables, since their values (or even their types) might change at almost any time. If a global variable will not change, adding a const declaration solves this performance problem.

## Summary
- Modules and packages help with structure large code bases.
- Writing pretty code is a good thing
    - see also the official [style guide](https://docs.julialang.org/en/v1/manual/style-guide/)